This page describes the model setup, initial conditions, and basic output processing for the rotating_gravity_current test case within MOM6-examples.

_Written by Elizabeth Yankovsky, June 2019. For any questions please contact eyankovsky@gmail.com._
***
# Overview
The rotating gravity current experiment is meant to simulate an idealized dense gravity current or overflow. A shallow continental shelf region undergoes constant in time, negative buoyancy forcing comprised of a heat flux (due to atmospheric cooling) and an evaporative flux (due to ice formation). Both are directed out of the ocean, creating dense water on the shelf. This may simulate dense shelf overflows in the Arctic, where cooling combined with ice formation in the fall and winter months creates a buildup of highly dense waters in the shallow continental shelf seas. The forcing, initial conditions, and topography of this test case may be modified to simulate Antarctic coastal polynyas or other shelf overflows around the world. The experiment is carried out for 60 days, by which point a steady state is achieved. The experiment assumes an f-plane, causing the gravity current to be influenced by rotation. The main dynamics involve:
1. The creation of a bottom-intensified and compensating surface-intensified geostrophic jet as the dense water moves offshore along the bottom and return flow flows into the shelf near the surface. 
2. Breaking of geostrophy by bottom friction and symmetric instability in the 2D cases, development of baroclinic instability in the 3D case.
3. Dissipation of geostrophic energy and mixing by various mechanisms including symmetric instability, eddy-driven stirring combined with viscous effects, parameterized shear mixing (Jackson shear mixing parameterization is employed), and numerical mixing in the Z\* coordinates.
The test case allows one to examine how different mixing parameterizations, resolutions, and coordinate systems influence representation of rotating gravity current dynamics and instability dynamics.

There are four simulation directories in this test case: `Layersetup_2D`, `Layersetup_3D`, `Zsetup_2D`, and `Zsetup_3D`. All cases are identical with the exception of the coordinate system (Layer coordinates in the first two and Z\* ALE coordinates in the second two) and dimension. There is also a directory titled `Initial_Conditions` containing IPython notebooks for generating the initial conditions. There are notebooks for the 2D and 3D topography files (identical for Z\* and Layer coordinates) titled `2D Initial conditions - topography.ipynb` and `3D Initial conditions - topography.ipynb`. Forcing is also prescribed identically in Z\* and Layer coordinates, the notebooks for 2D and 3D are `2D Initial conditions - forcing.ipynb` and `3D Initial conditions - forcing.ipynb`. Finally, there is a notebook for generating initial temperature, salinity, and coordinate files for each case: `Initial conditions - 2D Z*.ipynb`, `Initial conditions - 2D Layer.ipynb`, `Initial conditions - 3D Z*.ipynb`, and `Initial conditions - 3D Layer.ipynb`. These notebooks are written so that a 1D NetCDF file is first read, and then the initial conditions are created from that. Alternatively, one can define all of the initial conditions analytically without reading in existing data. Within the simulation directories there are also notebooks for basic plotting of the output data, titled `Analysis -- xz fields.ipynb` or `Analysis -- xy fields.ipynb`.

Below, we will examine how to create initial conditions for both Z\* and Layer coordinates (slightly more complicated in the latter), how to modify MOM_input and other runtime features, and how to handle output and restarts. 
***

# Initial Conditions
The initial conditions consist of: topography, forcing values, and temperature and salinity fields. For the Layer case, we must also specify density coordinates and initial layer thicknesses. The topography and forcing files are the same regardless of coordinate system, differences arise in the T and S initial conditions. All of the input files are NetCDF files that should be stored in the `INPUT/` directory within each simulation directory. To see the contents of a NetCDF file, execute `ncdump -h filename.nc`.
### Topography
To define the topography, we first specify the following in the MOM_input file:
```
TOPO_CONFIG = "file"             !
TOPO_FILE = "topog.nc"           ! default = "topog.nc"
                                 ! The file from which the bathymetry is read.
TOPO_VARNAME = "depth"           ! default = "depth"`
                                 ! The name of the bathymetry variable in TOPO_FILE.
```
Use the provided IPython notebook to generate the `topog.nc` file for the 2D cases (or `topog3D.nc` file for the 3D cases). The resulting 2D `topog.nc` file should contain the following:
```
ncdump -h topog.nc
netcdf topog {
dimensions:
	ntiles = 1 ;
	nx = 640 ;
	ny = 4 ;
variables:
	double nx(nx) ;
		nx:standard_name = "x location at cell centers" ;
		nx:units = "km" ;
	double ny(ny) ;
		ny:standard_name = "y location at cell center" ;
		ny:units = "km" ;
	double depth(ny, nx) ;
		depth:standard_name = "topographic depth at cell centers" ;
		depth:units = "meters" ;
}
```
The values of depth for this case range from 40 to 2500 meters (and are all positive). Note that even though this a 2D case (640 points in the x direction), we still have 4 points in y - this is because the minimum number of halo points (specified in MOM_input) is 4.

### Forcing
To define the buoyancy forcing, we first specify the following in the MOM_input file:
```
BUOY_CONFIG = "file"            !
                                ! The character string that indicates how buoyancy forcing
                                ! is specified. Valid options include (file), (zero),
                                ! (linear), (USER), and (NONE).
```
Then, a file containing all the forcing fields must be specified for each of the forcing fields as shown. In this case, all the fields are set to zero within the forcing file with the exception of the sensible heat flux and evaporation. Additionally, this file must contain a reference surface salinity and temperature.
```
LONGWAVE_FILE = "forcing.nc"    !
                                ! The file with the longwave heat flux, in the variable
                                ! given by LONGWAVE_FORCING_VAR.
SHORTWAVE_FILE = "forcing.nc"   !
                                ! The file with the shortwave heat flux, in the variable
                                ! given by SHORTWAVE_FORCING_VAR.
EVAPORATION_FILE = "forcing.nc" !
                                ! The file with the evaporative moisture flux, in the
                                ! variable given by EVAP_FORCING_VAR.
LATENTHEAT_FILE = "forcing.nc"  !
                                ! The file with the latent heat flux, in the variable
                                ! given by LATENT_FORCING_VAR.
SENSIBLEHEAT_FILE = "forcing.nc" !
                                ! The file with the sensible heat flux, in the variable
                                ! given by SENSIBLE_FORCING_VAR.
RAIN_FILE = "forcing.nc"        !
                                ! The file with the liquid precipitation flux, in the
                                ! variable given by RAIN_FORCING_VAR.
SNOW_FILE = "forcing.nc"        !
                                ! The file with the frozen precipitation flux, in the
                                ! variable given by SNOW_FORCING_VAR.
RUNOFF_FILE = "forcing.nc"      !
                                ! The file with the fresh and frozen runoff/calving
                                ! fluxes, in variables given by LIQ_RUNOFF_FORCING_VAR
                                ! and FROZ_RUNOFF_FORCING_VAR.
SSTRESTORE_FILE = "forcing.nc"  !
                                ! The file with the SST toward which to restore in the
                                ! variable given by SST_RESTORE_VAR.
SALINITYRESTORE_FILE = "forcing.nc" !
                                ! The file with the surface salinity toward which to
                                ! restore in the variable given by SSS_RESTORE_VAR.
```

If we examine the contents of `forcing.nc` (again, should be placed into the `INPUT/` directory of the simulation):
```
ncdump -h forcing.nc
netcdf forcing {
dimensions:
	TIME = UNLIMITED ; // (1 currently)
	LAT = 4 ;
	LON = 640 ;
variables:
	double TIME(TIME) ;
		TIME:standard_name = "time" ;
		TIME:units = "days since 0001-01-01 00:00:00" ;
	double evap(TIME, LAT, LON) ;
		evap:standard_name = "Salt flux forcing = evaporation (both negative)" ;
		evap:units = "kg/(m^2*s)" ;
	double LW(TIME, LAT, LON) ;
		LW:standard_name = "Longwave flux downward" ;
		LW:units = "W/m^2" ;
	double SW(TIME, LAT, LON) ;
		SW:standard_name = "Shortwave flux downward" ;
		SW:units = "W/m^2" ;
	double sensible(TIME, LAT, LON) ;
		sensible:standard_name = "Sensible heat flux into ocean" ;
		sensible:units = "W/m^2" ;
	double latent(TIME, LAT, LON) ;
		latent:standard_name = "Latent heat flux into ocean" ;
		latent:units = "W/m^2" ;
	double liq_precip(TIME, LAT, LON) ;
		liq_precip:standard_name = "Liquid precipitation into ocean" ;
		liq_precip:units = "kg/(m^2*s)" ;
	double froz_precip(TIME, LAT, LON) ;
		froz_precip:standard_name = "Frozen precipitation into ocean" ;
		froz_precip:units = "kg/(m^2*s)" ;
	double liq_runoff(TIME, LAT, LON) ;
		liq_runoff:standard_name = "Liquid runoff into ocean" ;
		liq_runoff:units = "kg/(m^2*s)" ;
	double froz_runoff(TIME, LAT, LON) ;
		froz_runoff:standard_name = "Frozen runoff into ocean" ;
		froz_runoff:units = "kg/(m^2*s)" ;
	double SST(TIME, LAT, LON) ;
		SST:standard_name = "Reference surface SST" ;
		SST:units = "Celsius" ;
	double SSS(TIME, LAT, LON) ;
		SSS:standard_name = "Reference surface SSS" ;
		SSS:units = "psu" ;
}
```
This `forcing.nc` file (or `forcing3D.nc` for the 3D simulations) is created using the provided IPython notebooks. 

Again, the topography and forcing files are generated in the same way regardless of whether we are in Z* or Layer coordinates. We now move on to setting up the initial temperature, salinity, and coordinates. This is done differently based on which coordinate system we are employing.

## Z\* Coordinates
To define initial conditions in Z\* coordinates from an initial grid in pure Z space, we modify MOM_input as follows:
```
COORD_CONFIG = "linear"              !
THICKNESS_CONFIG = "uniform"         !
INIT_LAYERS_FROM_Z_FILE = True       ! Doing this instead of TS_CONFIG="file" and TS_FILE="filename"
TEMP_SALT_Z_INIT_FILE = "IC_TS.nc"   ! Initial conditions file
Z_INIT_FILE_PTEMP_VAR = "PTEMP"      ! Variable name for potential temperature
Z_INIT_FILE_SALT_VAR = "SALT"        ! Variable name for salt
Z_INIT_ALE_REMAPPING = True          !
Z_INIT_REMAP_OLD_ALG = False         !
REMAP_UV_USING_OLD_ALG = False       !
```
The `IC_TS.nc` file is created using the notebooks `Initial conditions - 2D Z*.ipynb` or `Initial conditions - 3D Z*.ipynb`, for 2D and 3D Z\* simulations respectively. `IC_TS.nc` should be stored in the simulation `INPUT/` directory and its header should appear as follows:
```
ncdump -h IC_TS.nc
netcdf IC_TS {
dimensions:
	TIME = 1 ;
	DEPTH = 120 ;
	LAT = 4 ;
	LON = 640 ;
variables:
	double LAT(LAT) ;
		LAT:standard_name = "YC (y coordinate at center of grid cell)" ;
		LAT:units = "km" ;
		LAT:cartesian_axis = "Y" ;
	double LON(LON) ;
		LON:standard_name = "XC (x coordinate at center of grid cell)" ;
		LON:units = "km" ;
		LON:cartesian_axis = "X" ;
	double TIME(TIME) ;
		TIME:standard_name = "time" ;
		TIME:units = "days since 0001-01-01 00:00:00" ;
		TIME:calendar = "noleap" ;
		TIME:cartesian_axis = "T" ;
	double DEPTH(DEPTH) ;
		DEPTH:standard_name = "Z coordinate" ;
		DEPTH:units = "m" ;
		DEPTH:cartesian_axis = "Z" ;
	float PTEMP(TIME, DEPTH, LAT, LON) ;
		PTEMP:_FillValue = -1.e+34f ;
		PTEMP:standard_name = "Initial potential temperature referenced to 0dbar" ;
		PTEMP:units = "Celsius" ;
	float SALT(TIME, DEPTH, LAT, LON) ;
		SALT:_FillValue = -1.e+34f ;
		SALT:standard_name = "Initial salinity" ;
		SALT:units = "PSU" ;
}
```
## Layer Coordinates
Defining initial temperature and salinity conditions as well as setting up the coordinates in Layer mode is slightly more challenging than Z\*, since the coordinates are densities rather than depths. Here are the relevant parameters in MOM_input:
```
COORD_CONFIG = "ts_profile"     !
                                ! This specifies how layers are to be defined:
                                !     file - read coordinate information from the file
                                !       specified by (COORD_FILE).
                                !     linear - linear based on interfaces not layesrs.
                                !     ts_ref - use reference temperature and salinity
                                !     ts_range - use range of temperature and salinity
                                !       (T_REF and S_REF) to determine surface density
                                !       and GINT calculate internal densities.
                                !     gprime - use reference density (RHO_0) for surface
                                !       density and GINT calculate internal densities.
                                !     ts_profile - use temperature and salinity profiles
                                !       (read from COORD_FILE) to set layer densities.
                                !     USER - call a user modified routine.
COORD_FILE = "Coord_file.nc"    ! Temperature and salinity coordinates (1D arrays)
P_REF = 0.0E+07                 !   [Pa] default = 2.0E+07
                                ! The pressure that is used for calculating the coordinate
                                ! density.  (1 Pa = 1e4 dbar, so 2e7 is commonly used.)
                                ! This is only used if USE_EOS and ENABLE_THERMODYNAMICS
                                ! are true.

THICKNESS_CONFIG = "file"       ! A string that determines how the initial layer
                                ! thicknesses are specified for a new run
THICKNESS_FILE="thickness_file.nc" ! Contains layer thicknesses h, not corrected for topography!
ADJUST_THICKNESS=TRUE           ! Corrects thicknesses for topography
TS_CONFIG = "file"              !
TS_FILE = "IC_TS.nc"            ! Initial condition file for T and S.
FIT_TO_TARGET_DENSITY_IC = False!   [Boolean] default = True
REMAP_UV_USING_OLD_ALG = False  !
```
All three input files (`thickness_file.nc`, `IC_TS.nc`, and `Coord_file.nc`) are generated by the notebook `Initial conditions - 2D Layer.ipynb` for the 2D case. They should all be placed inside the `INPUT/` directory. Here is what this setup accomplishes:
1. We define the density coordinates for the model using the file `Coord_file.nc`. This contains a 1D list of temperatures and salinities that represent the range of temperatures and salinities for the density coordinates. Each pair of values in TS-space is used with the reference pressure (`P_REF`) to calculate a density. The resulting densities are the coordinates of the model. Keep in mind that we have negative buoyancy forcing on the shelf that creates highly dense waters that are not present in the initial conditions we used in Z\*. Therefore, the  model coordinates should not match the initial conditions that we used in Z\* space. It is advisable to first run the model in Z\* coordinates, see what the final TS range is, and apply that knowledge to defining the coordinates. Otherwise, the resolution of the highly dense waters will be very poor. The header of `Coord_file.nc` should appear as:
```
ncdump -h Coord_file.nc
netcdf Coord_file {
dimensions:
	TIME = 1 ;
	DEPTH = 120 ;
variables:
	float PTEMP(DEPTH) ;
		PTEMP:_FillValue = -1.e+34f ;
		PTEMP:standard_name = "Layer temperature referenced to 0dbar" ;
		PTEMP:units = "Celsius" ;
	float SALT(DEPTH) ;
		SALT:_FillValue = -1.e+34f ;
		SALT:standard_name = "Layer salinity" ;
		SALT:units = "PSU" ;
}
```
2. We define the initial conditions in the file `IC_TS.nc`. This contains 3D arrays of the initial temperature and salinity values, but not the thicknesses of these layers (done in the next step). This is different than the Z\* case because in this initial condition file we will include higher density layers that are initially unfilled (we will give them very small thicknesses in the next step). The header of `IC_TS.nc` should appear as:
```
ncdump -h IC_TS.nc
netcdf IC_TS {
dimensions:
	TIME = 1 ;
	LAYER = 120 ;
	LAT = 4 ;
	LON = 640 ;
variables:
	double LAT(LAT) ;
		LAT:standard_name = "YC (y coordinate at center of grid cell)" ;
		LAT:units = "km" ;
		LAT:cartesian_axis = "Y" ;
	double LON(LON) ;
		LON:standard_name = "XC (x coordinate at center of grid cell)" ;
		LON:units = "km" ;
		LON:cartesian_axis = "X" ;
	double TIME(TIME) ;
		TIME:standard_name = "time" ;
		TIME:units = "days since 0001-01-01 00:00:00" ;
		TIME:calendar = "noleap" ;
		TIME:cartesian_axis = "T" ;
	float PTEMP(TIME, LAYER, LAT, LON) ;
		PTEMP:_FillValue = -1.e+34f ;
		PTEMP:standard_name = "Initial potential temperature referenced to 0dbar" ;
		PTEMP:units = "Celsius" ;
	float SALT(TIME, LAYER, LAT, LON) ;
		SALT:_FillValue = -1.e+34f ;
		SALT:standard_name = "Initial salinity" ;
		SALT:units = "PSU" ;
}
```
3. We have specified the initial temperature and salinities of the layers - now we must specify how thick each layer is (in meters). This is done in `thickness_file.nc`. Again, some layers are initially unfilled, as we have negative buoyancy forcing on the shelf that creates highly dense waters. We therefore initialize the model to have some layers that are initially infinitesimally thin. The parameter `ADJUST_THICKNESS` adjusts all the layers to take into account the topography. 

# Sponge and Tracer
This experiment is designed to have its own packages for a passive tracer and an offshore sponge. The code for the tracer can be viewed in `MOM6-examples/src/MOM6/src/tracer/RGC_tracer.F90`. The tracer is a passive dye that tags water in the forcing region, tracing it as it propagates offshore and diluting through mixing. Values of the tracer are set to 1.0 at the surface of the forcing region at each time step, and damped to zero within the sponge boundary offshore. The sponge boundary code is located in `MOM6-examples/src/MOM6/src/user/RGC_initialization.F90`. In MOM_input, the following need to be added:
```
SPONGE = True                   !   [Boolean] default = False
                                ! If true, sponges may be applied anywhere in the domain.
                                ! The exact location and properties of those sponges are
                                ! specified via SPONGE_CONFIG.
SPONGE_CONFIG = "RGC"           ! default = "file"
                                ! A string that sets how the sponges are configured:
                                !     file - read sponge properties from the file
                                !       specified by (SPONGE_FILE).
                                !     ISOMIP - apply ale sponge in the ISOMIP case
                                !     IDEAL_IS - apply sponge in the IDEAL_IS case
                                !     DOME - use a slope and channel configuration for the
                                !       DOME sill-overflow test case.
                                !     BFB - Sponge at the southern boundary of the domain
                                !       for buoyancy-forced basin case.
                                !     USER - call a user modified routine.
RGC_TNUDG = 10.0                ! default = 0.0
                                ! Nudging time scale for sponge layers (days)
RGC_SPONGE_FILE = "sponge_IC.nc" !
                                ! The name of the file with temps., salts. and interfaces to
                                !  damp toward.
SPONGE_PTEMP_VAR = "Temp"       ! default = "Temp"
                                ! The name of the potential temperature variable in
                                ! SPONGE_STATE_FILE.
SPONGE_SALT_VAR = "Salt"        ! default = "Salt"
                                ! The name of the salinity variable in
                                ! SPONGE_STATE_FILE.
SPONGE_ETA_VAR = "eta"          ! default = "eta"
                                ! The name of the interface height variable in
                                ! SPONGE_STATE_FILE.
SPONGE_H_VAR = "h"              ! default = "h"
                                ! The name of the layer thickness variable in
                                ! SPONGE_STATE_FILE.
SPONGE_UV = True                !   [Boolean] default = False
                                ! Apply sponges in u and v, in addition to tracers.
USE_RGC_TRACER = True           ! Using RGC passive tracer configuration.
CONT_SHELF_LENGTH = 15.0        ! Where tracer is set to 1.0 at surface.
```
In order to turn on the sponge, we see that we need the file `sponge_IC.nc`. This is created by running the model with the sponge turned off for one time step. The model will generate an `Initial_state.nc` file which should be renamed to `sponge_IC.nc` and placed within the `INPUT/` directory. Then, the sponge can be turned on (TRUE) and as the model runs it will damp tracers and velocities within the offshore boundary to the initial conditions. The boundary conditions in y are periodic (`REENTRANT_Y = True` in MOM_input), the sponge is only applied offshore in x. 

# Restarts and Output
There are many additional runtime parameters that may be manipulated within MOM_input. The complete list of parameters and their defaults that may be added to MOM_input is located in MOM_parameter_doc.all. If one of the parameters from MOM_parameter_doc.all is not added to MOM_input, the default is used. Examples of parameters that may be modified at runtime include time steps, domain size, viscosity, mixing parameterization choice, boundary conditions, and length of the simulation. In this experiment we set `TIMEUNIT = 3600.0`, which defines the unit of time as one hour (3600 seconds). We then set `DAYMAX = 1440.0`, which is the length of the simulation in terms of time units. 1440.0 hours is 60 days - the length of the simulation. If we want to restart the simulation, we specify `RESTINT = 1440.0`, which will save a restart file at 1440.0 hours into the `RESTART/` directory. To restart the simulation from 60 days, we do the following:
1. Copy the contents out of the `RESTART/` directory into the `INPUT/` directory.
2. Save all the existing output files from the first 60 days or rename them, otherwise they will be overwritten. 
3. Edit `input.nml`, changing `input_filename = 'n'` to `input_filename = 'r'`. ('n' stands for 'new', 'r' stands for 'restart'.)
4. To run for another 1440.0 time units (60 days), edit MOM_input to have DAYMAX = 2880.0. Run the simulation (submit `runscript2D` or `runscript3D`)

To control which diagnostics are saved and at what frequency, edit the `diag_table` file. Presently, the experiment is set up to save both Z\* or Layer coordinate output as well as remapped Z coordinate output. The former is saved into a `prog.nc` file, the latter into `prog_z.nc`. Output is saved every 6 hours, and not time averaged. The list of all the available diagnostics is in `available_diags.000000`. There are choices corresponding to `ocean_model` (native coordinate output) and `ocean_model_z` (remapped onto Z output).